This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import display
from pydrake.all import (
    MakeVectorContinuousVariable,
    MathematicalProgram,
    RealContinuousLyapunovEquation,
    RegionOfAttraction,
    Solve,
    SymbolicVectorSystem,
    Variable,
    Variables,
    plot_sublevelset_expression,
)
from pydrake.examples import VanDerPolOscillator

from underactuated import plot_2d_phase_portrait, running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

# Time-reversed van der Pol Oscillator

We also use this example in the exercises at the end of the Lyapunov chapter, and work through all of the details of the formulation.  I highly recommend you try it out!

N.B. -- we know how to get much larger/tighter (inner) approximations of this RoA.  I will be implementing those ideas in drake's RegionOfAttraction method very soon.  Hopefully the region will be even bigger next time you try it.

In [ ]:
def vdp_roa():
    x1 = Variable("x1")
    x2 = Variable("x2")
    sys = SymbolicVectorSystem(state=[x1, x2], dynamics=[-x2, x1 + (x1 * x1 - 1) * x2])
    context = sys.CreateDefaultContext()
    V = RegionOfAttraction(system=sys, context=context)

    fig, ax = plt.subplots(figsize=(10, 10))
    plot_2d_phase_portrait(sys, (-3, 3), (-3, 3))
    limit_cycle = VanDerPolOscillator.CalcLimitCycle()
    plt.plot(
        limit_cycle[0],
        limit_cycle[1],
        color="k",
        linewidth=3,
        label="Known ROA boundary",
    )
    plt.legend(loc=1)
    plot_sublevelset_expression(ax, V)
    display(mpld3.display())


vdp_roa()

# Searching for the Lyapunov function

The previous example used the linearization to propose a candidate Lyapunov function, and SOS optimization to verify the largest possible sublevel set of that function. We can do better if allow the optimization to also change the parameters of the Lyapunov function. But this problem is non-convex, so we use bilinear alternations.

In [ ]:
# TODO(russt): Generalize this example to non-quadratic Lyapunov functions.
# TODO(russt): Improve the numerics (e.g. BalanceQuadraticForms, etc)

from pydrake.all import MosekSolver, ScsSolver


def vdp_roa_optimize_quadratic():
    # Clarabel v0.9.0 fails to solve this problem.
    # See https://github.com/oxfordcontrol/Clarabel.rs/issues/119
    solver = (
        MosekSolver()
        if (MosekSolver().available() and MosekSolver().enabled())
        else ScsSolver()
    )

    # function that implements the time-reversed Van der Pol dynamics
    f = lambda x: [-x[1], x[0] + (x[0] ** 2 - 1) * x[1]]

    def OptimizeMultipliers(x, P, lambda_degree=4):
        prog = MathematicalProgram()
        prog.AddIndeterminates(x)

        V = x.dot(P).dot(x)
        Vdot = V.Jacobian(x).dot(f(x))

        l = prog.NewFreePolynomial(Variables(x), lambda_degree).ToExpression()
        rho = prog.NewContinuousVariables(1, "rho")[0]

        prog.AddSosConstraint(x.dot(x) * (V - rho) - l * Vdot)

        prog.AddLinearCost(-rho)

        result = solver.Solve(prog)
        assert result.is_success()

        return result.GetSolution(l), P / result.GetSolution(rho)

    def OptimizeLyapunov(x, Phat, l):
        prog = MathematicalProgram()
        prog.AddIndeterminates(x)
        P = prog.NewSymmetricContinuousVariables(2, "P")

        prog.AddLinearCost(np.trace(np.linalg.inv(Phat) @ P))

        V = x.dot(P).dot(x)
        Vdot = V.Jacobian(x).dot(f(x))

        prog.AddSosConstraint(x.dot(x) * (V - 1) - l * Vdot)
        prog.AddPositiveSemidefiniteConstraint(P)

        result = solver.Solve(prog)
        assert result.is_success()

        return result.GetSolution(P)

    x = MakeVectorContinuousVariable(2, "x")
    A = np.array([[0, -1], [1, -1]])
    Q = np.eye(2)
    P = RealContinuousLyapunovEquation(A, Q)
    print(f"P = {P}")
    l, P = OptimizeMultipliers(x, P)
    P_last = np.eye(2)
    while np.linalg.norm(P - P_last) > 1e-3:
        P_last = P
        P = OptimizeLyapunov(x, P, l)
        l, P = OptimizeMultipliers(x, P)
        print(f"P = {P}")

    fig, ax = plt.subplots(figsize=(10, 10))
    sys = SymbolicVectorSystem(state=x, dynamics=f(x))
    plot_2d_phase_portrait(sys, (-3, 3), (-3, 3))
    limit_cycle = VanDerPolOscillator.CalcLimitCycle()
    plt.plot(
        limit_cycle[0],
        limit_cycle[1],
        color="k",
        linewidth=3,
        label="Known ROA boundary",
    )
    plt.legend(loc=1)
    V = x.dot(P).dot(x)
    plot_sublevelset_expression(ax, V)
    display(mpld3.display())


vdp_roa_optimize_quadratic()